In [25]:
import os
import json

In [26]:
# load json file and read line by line
def load_json_file(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [27]:
data_part = "all"
train_path = f"/research/d5/gds/xywen22/project/llm_framework/AST_multiwoz_part/data/processed/train_AST_multiwoz_woaction_{data_part}.json"

train_data = load_json_file(train_path)

In [28]:
print(train_data[0]['target'])
print(train_data[1]['target'])

find_hotel [with parking, cheap, with internet, alexander bed and breakfast, 4 stars]
book_hotel [saturday, 6 people, 5 stay]


#### process the actions described in the format of natural language to the structured format

In [29]:
mapping_dict = {
  "find_hotel": "search for hotel",
  "book_hotel": "book hotel",
  "find_train": "search for trains",
  "book_train": "book train ticket",
  "find_attraction": "search for attractions",
  "find_restaurant": "search for restaurants",
  "book_restaurant": "book table at restaurant",
  "find_hospital": "search for hospital",
  "book_taxi": "book taxi",
  "find_taxi": "search for taxi",
  "find_bus": "search for bus",
  "find_police": "search for police station"
}

# conver key to value and value to key
def convert_dict(mapping_dict):
    new_dict = {}
    for key, value in mapping_dict.items():
        new_dict[value] = key
    return new_dict

new_mapping_dict = convert_dict(mapping_dict)
print(new_mapping_dict)

{'search for hotel': 'find_hotel', 'book hotel': 'book_hotel', 'search for trains': 'find_train', 'book train ticket': 'book_train', 'search for attractions': 'find_attraction', 'search for restaurants': 'find_restaurant', 'book table at restaurant': 'book_restaurant', 'search for hospital': 'find_hospital', 'book taxi': 'book_taxi', 'search for taxi': 'find_taxi', 'search for bus': 'find_bus', 'search for police station': 'find_police'}


In [30]:
all_transitions = []
tmp_all_transitions = {}
# for i in range(len(train_data)):
#     # if i == 10:
#     #     break
#     # print(new_mapping_dict[train_data[i]['target']])
#     target_pieces = train_data[i]['target'].split(';')
#     # pull up the costumer account [alessandro phoenix]
#     # only get the first part before the first '['
#     for j in range(len(target_pieces)):
#         target_pieces[j] = target_pieces[j].split('[')[0].strip()
    
#     for j in range(len(target_pieces)):
#         if target_pieces[j] in new_mapping_dict:
#             target_pieces[j] = new_mapping_dict[target_pieces[j]]
#         else:
#             print("not found")

#     # print(target_pieces)

#     all_transitions.append(target_pieces)

for i in range(len(train_data)):
    action = train_data[i]['target'].split(' ')[0].strip()
    if train_data[i]['convo_id'] not in tmp_all_transitions:
        tmp_all_transitions[train_data[i]['convo_id']] = []
    tmp_all_transitions[train_data[i]['convo_id']].append(action)

for key, value in tmp_all_transitions.items():
    all_transitions.append(value)

print("all_transitions: ", all_transitions)

# count the freequency of the first action in each dialogue
action_count = {}
for i in range(len(all_transitions)):
    if all_transitions[i][0] not in action_count:
        action_count[all_transitions[i][0]] = 1
    else:
        action_count[all_transitions[i][0]] += 1

print("action_count: ", json.dumps(action_count, indent=2))

# convert to rate
action_rate = {}
total_dialogue = len(all_transitions)
for key, value in action_count.items():
    action_rate[key] = value / total_dialogue

print("action_rate: ", json.dumps(action_rate, indent=2))

all_transitions:  [['find_hotel', 'book_hotel', 'find_train', 'book_train'], ['find_train', 'book_train', 'find_restaurant'], ['find_restaurant'], ['find_hotel', 'find_restaurant', 'book_restaurant'], ['find_restaurant', 'book_restaurant'], ['find_train', 'find_hotel'], ['find_taxi'], ['find_attraction'], ['find_train', 'find_attraction', 'find_restaurant'], ['find_hospital'], ['find_attraction', 'find_train', 'book_train'], ['find_taxi'], ['find_train'], ['find_train', 'find_restaurant', 'book_train', 'book_restaurant'], ['find_attraction', 'find_hotel'], ['find_police'], ['find_train', 'find_attraction', 'book_train'], ['find_restaurant'], ['find_restaurant', 'book_restaurant', 'find_attraction'], ['find_restaurant'], ['find_hotel', 'book_hotel', 'find_train', 'book_train'], ['find_restaurant', 'find_hotel'], ['find_attraction', 'find_hotel', 'book_hotel'], ['find_restaurant', 'find_train'], ['find_train', 'find_hotel', 'book_hotel'], ['find_train', 'find_hotel', 'book_train', 'book_

In [31]:
possible_states = mapping_dict.keys()
print(list(possible_states))
possible_states = list(possible_states)
# ['pull-up-account', 'enter-details', 'verify-identity', 'make-password', 'search-timing', 'search-policy', 'validate-purchase', 'search-faq', 'membership', 'search-boots', 'try-again', 'ask-the-oracle', 'update-order', 'promo-code', 'update-account', 'search-membership', 'make-purchase', 'offer-refund', 'notify-team', 'record-reason', 'search-jeans', 'shipping-status', 'search-shirt', 'instructions', 'search-jacket', 'log-out-in', 'select-faq', 'subscription-status', 'send-link', 'search-pricing']

['find_hotel', 'book_hotel', 'find_train', 'book_train', 'find_attraction', 'find_restaurant', 'book_restaurant', 'find_hospital', 'book_taxi', 'find_taxi', 'find_bus', 'find_police']


In [32]:
# count the transition frequency between every two actions/state according to all_transitions
transition_count = {}
for i in range(len(all_transitions)):
    for j in range(len(all_transitions[i]) - 1):
        if (all_transitions[i][j], all_transitions[i][j+1]) in transition_count:
            transition_count[(all_transitions[i][j], all_transitions[i][j+1])] += 1
        else:
            transition_count[(all_transitions[i][j], all_transitions[i][j+1])] = 1

print("transition_count: ", transition_count)

# if no transition between two states, set the frequency to 0
for state1 in possible_states:
    for state2 in possible_states:
        if (state1, state2) not in transition_count:
            transition_count[(state1, state2)] = 0

print("transition_count: ", transition_count)

transition_count:  {('find_hotel', 'book_hotel'): 1585, ('book_hotel', 'find_train'): 201, ('find_train', 'book_train'): 1288, ('book_train', 'find_restaurant'): 96, ('find_hotel', 'find_restaurant'): 350, ('find_restaurant', 'book_restaurant'): 1470, ('find_train', 'find_hotel'): 386, ('find_train', 'find_attraction'): 248, ('find_attraction', 'find_restaurant'): 422, ('find_attraction', 'find_train'): 389, ('find_train', 'find_restaurant'): 301, ('find_restaurant', 'book_train'): 125, ('book_train', 'book_restaurant'): 115, ('find_attraction', 'find_hotel'): 447, ('find_attraction', 'book_train'): 35, ('book_restaurant', 'find_attraction'): 94, ('find_restaurant', 'find_hotel'): 314, ('find_restaurant', 'find_train'): 244, ('find_hotel', 'book_train'): 123, ('book_train', 'book_hotel'): 96, ('find_taxi', 'find_attraction'): 77, ('find_hotel', 'find_attraction'): 232, ('find_attraction', 'find_taxi'): 249, ('book_hotel', 'find_attraction'): 181, ('book_restaurant', 'find_taxi'): 501, 

In [33]:
# print the pairs number
counter = 0
for key, value in transition_count.items():
    counter += 1
    print(key, value)

print("counter: ", counter)

('find_hotel', 'book_hotel') 1585
('book_hotel', 'find_train') 201
('find_train', 'book_train') 1288
('book_train', 'find_restaurant') 96
('find_hotel', 'find_restaurant') 350
('find_restaurant', 'book_restaurant') 1470
('find_train', 'find_hotel') 386
('find_train', 'find_attraction') 248
('find_attraction', 'find_restaurant') 422
('find_attraction', 'find_train') 389
('find_train', 'find_restaurant') 301
('find_restaurant', 'book_train') 125
('book_train', 'book_restaurant') 115
('find_attraction', 'find_hotel') 447
('find_attraction', 'book_train') 35
('book_restaurant', 'find_attraction') 94
('find_restaurant', 'find_hotel') 314
('find_restaurant', 'find_train') 244
('find_hotel', 'book_train') 123
('book_train', 'book_hotel') 96
('find_taxi', 'find_attraction') 77
('find_hotel', 'find_attraction') 232
('find_attraction', 'find_taxi') 249
('book_hotel', 'find_attraction') 181
('book_restaurant', 'find_taxi') 501
('book_hotel', 'find_restaurant') 160
('find_restaurant', 'find_taxi')

In [34]:
# count the frequency of each state
transition_matrix = {}
for state1 in possible_states:
    transition_matrix[state1] = {}
    for state2 in possible_states:
        transition_matrix[state1][state2] = 0
    
    for key, value in transition_count.items():
        if state1 == key[0]:
            transition_matrix[state1][key[1]] = value

print("transition_matrix: ", transition_matrix)
print(len(transition_matrix['find_restaurant']))

# convert count to probability
for state1 in possible_states:
    sum_count = sum(transition_matrix[state1].values())
    for state2 in possible_states:
        if sum_count != 0:
            transition_matrix[state1][state2] = transition_matrix[state1][state2] / sum_count
        
print("transition_matrix: ", transition_matrix)
print(len(transition_matrix['find_restaurant']))
print("sum of probability: ", sum(transition_matrix['find_restaurant'].values()))
print("sum of probability: ", sum(transition_matrix['find_train'].values()))

transition_matrix:  {'find_hotel': {'find_hotel': 0, 'book_hotel': 1585, 'find_train': 224, 'book_train': 123, 'find_attraction': 232, 'find_restaurant': 350, 'book_restaurant': 162, 'find_hospital': 17, 'book_taxi': 0, 'find_taxi': 100, 'find_bus': 0, 'find_police': 0}, 'book_hotel': {'find_hotel': 0, 'book_hotel': 0, 'find_train': 201, 'book_train': 49, 'find_attraction': 181, 'find_restaurant': 160, 'book_restaurant': 101, 'find_hospital': 2, 'book_taxi': 0, 'find_taxi': 227, 'find_bus': 0, 'find_police': 0}, 'find_train': {'find_hotel': 386, 'book_hotel': 83, 'find_train': 0, 'book_train': 1288, 'find_attraction': 248, 'find_restaurant': 301, 'book_restaurant': 75, 'find_hospital': 3, 'book_taxi': 0, 'find_taxi': 15, 'find_bus': 1, 'find_police': 2}, 'book_train': {'find_hotel': 198, 'book_hotel': 96, 'find_train': 0, 'book_train': 0, 'find_attraction': 225, 'find_restaurant': 96, 'book_restaurant': 115, 'find_hospital': 0, 'book_taxi': 0, 'find_taxi': 1, 'find_bus': 1, 'find_polic

In [35]:
# convert to mdp:
# here we use the dot format to represent the mdp
'''
digraph learned_mdp {
s0 [label="init"];
s1 [label="pull-up-account"];
s2 [label="end"];
s3 [label="enter-details"];
s0 -> s1  [label="pull-up-account:0.03125"];
s0 -> s3  [label="enter-details:0.03125"];
s0 -> s2  [label="end:0.03125"];
s1 -> s3  [label="pull-up-account:0.2202970297029703"];
s1 -> s1  [label="pull-up-account:0.7797029702970297"];
s1 -> s3  [label="enter-details:0.03125"];
s1 -> s2  [label="end:0.03125"];
s2 -> s1  [label="pull-up-account:0.03125"];
s2 -> s3  [label="enter-details:0.03125"];
s2 -> s2  [label="end:0.03125"];
s3 -> s1  [label="pull-up-account:0.03125"];
s3 -> s3  [label="enter-details:0.03125"];
s3 -> s2  [label="end:0.03125"];
__start0 [label="", shape=none];
__start0 -> s0  [label=""];
}
'''

# ['find_hotel', 'book_hotel', 'find_train', 'book_train', 'find_attraction', 'find_restaurant', 'book_restaurant', 'find_hospital', 'book_taxi', 'find_taxi', 'find_bus', 'find_police']

'''
action_rate:  {
  "find_hotel": 0.215,
  "find_train": 0.22875,
  "find_restaurant": 0.2875,
  "find_taxi": 0.0325,
  "find_attraction": 0.1675,
  "find_hospital": 0.03625,
  "find_police": 0.03,
  "book_hotel": 0.00125,
  "book_restaurant": 0.00125
}
'''

# convert transition_matrix to dot format
def convert_to_dot(transition_matrix):
    dot_str = "digraph learned_mdp {\n"
    dot_str += "s0 [label=\"init\"];\n"
    for i in range(len(possible_states)):
        dot_str += "s" + str(i+1) + " [label=\"" + possible_states[i] + "\"];\n"
    dot_str += "s13 [label=\"end\"];\n"    

    dot_str += "s0 -> s0  [label=\"init:1.0\"];\n"
    # dot_str += "s0 -> s1  [label=\"action:0.215\"];\n"
    # dot_str += "s0 -> s2  [label=\"action:0.00125\"];\n"
    # dot_str += "s0 -> s3  [label=\"action:0.22875\"];\n"
    # dot_str += "s0 -> s4  [label=\"action:0.0\"];\n"
    # dot_str += "s0 -> s5  [label=\"action:0.1675\"];\n"
    # dot_str += "s0 -> s6  [label=\"action:0.2875\"];\n"
    # dot_str += "s0 -> s7  [label=\"action:0.00125\"];\n"
    # dot_str += "s0 -> s8  [label=\"action:0.03625\"];\n"
    # dot_str += "s0 -> s9  [label=\"action:0.0\"];\n"
    # dot_str += "s0 -> s10  [label=\"action:0.0325\"];\n"
    # dot_str += "s0 -> s11  [label=\"action:0.0\"];\n"
    # dot_str += "s0 -> s12  [label=\"action:0.03\"];\n"
    for i in range(len(possible_states)):
        if possible_states[i] in action_rate:
            dot_str += "s0 -> s" + str(i+1) + "  [label=\"action:" + str(action_rate[possible_states[i]]) + "\"];\n"
        else:
            dot_str += "s0 -> s" + str(i+1) + "  [label=\"action:0.0\"];\n"
    dot_str += "s0 -> s13  [label=\"end:1\"];\n"

    for j in range(len(possible_states)):
        index = j
        state1 = possible_states[j]
        dot_str += "s" + str(index+1) + " -> s0  [label=\"init:1.0\"];\n"
        for k in range(len(possible_states)):
            index2 = k
            state2 = possible_states[k]
            if transition_matrix[state1][state2] != 0:
                dot_str += "s" + str(index+1) + " -> s" + str(index2+1) + "  [label=\"" + "action" + ":" + str(transition_matrix[state1][state2]) + "\"];\n"
        dot_str += 's' + str(index+1) + ' -> s13' + ' [label="end:1.0"];\n'
    
    dot_str += 's13 -> s13 [label="init:1.0"];\n'
    dot_str += 's13 -> s13  [label="action:1.0"];\n'
    dot_str += 's13 -> s13  [label="end:1.0"];\n'

    dot_str += "__start0 [label=\"\", shape=none];\n"
    dot_str += "__start0 -> s0  [label=\"\"];\n"
    dot_str += "}"

    return dot_str

dot_str = convert_to_dot(transition_matrix)
print(dot_str)

# write the dot string to a file
with open(f"learned_mdp_multiwoz_{data_part}.dot", "w") as f:
    f.write(dot_str)


digraph learned_mdp {
s0 [label="init"];
s1 [label="find_hotel"];
s2 [label="book_hotel"];
s3 [label="find_train"];
s4 [label="book_train"];
s5 [label="find_attraction"];
s6 [label="find_restaurant"];
s7 [label="book_restaurant"];
s8 [label="find_hospital"];
s9 [label="book_taxi"];
s10 [label="find_taxi"];
s11 [label="find_bus"];
s12 [label="find_police"];
s13 [label="end"];
s0 -> s0  [label="init:1.0"];
s0 -> s1  [label="action:0.22436885148749555"];
s0 -> s2  [label="action:0.0010667298802892024"];
s0 -> s3  [label="action:0.20374540713523764"];
s0 -> s4  [label="action:0.0011852554225435582"];
s0 -> s5  [label="action:0.1630911461419936"];
s0 -> s6  [label="action:0.30247718383311606"];
s0 -> s7  [label="action:0.0007111532535261348"];
s0 -> s8  [label="action:0.03377977954249141"];
s0 -> s9  [label="action:0.0"];
s0 -> s10  [label="action:0.04029868436648098"];
s0 -> s11  [label="action:0.0003555766267630674"];
s0 -> s12  [label="action:0.02892023231006282"];
s0 -> s13  [label="end

In [36]:
from aalpy.utils import load_automaton_from_file, save_automaton_to_file, visualize_automaton, generate_random_dfa
from aalpy.automata import Dfa
from aalpy.SULs import AutomatonSUL
from aalpy.oracles import RandomWalkEqOracle
from aalpy.learning_algs import run_Lstar, run_KV

# load an automaton
automaton = load_automaton_from_file(f"learned_mdp_multiwoz_{data_part}.dot", automaton_type='mdp')

print(automaton)

# visualize the automaton
visualize_automaton(automaton, path=f"learned_mdp_multiwoz_{data_part}.png")

digraph learnedModel {
s0 [label="init"];
s1 [label="find_hotel"];
s2 [label="book_hotel"];
s3 [label="find_train"];
s4 [label="book_train"];
s5 [label="find_attraction"];
s6 [label="find_restaurant"];
s7 [label="book_restaurant"];
s8 [label="find_hospital"];
s9 [label="book_taxi"];
s10 [label="find_taxi"];
s11 [label="find_bus"];
s12 [label="find_police"];
s13 [label="end"];
s0 -> s0  [label="init:1.0"];
s0 -> s1  [label="action:0.22"];
s0 -> s2  [label="action:0.0"];
s0 -> s3  [label="action:0.2"];
s0 -> s4  [label="action:0.0"];
s0 -> s5  [label="action:0.16"];
s0 -> s6  [label="action:0.3"];
s0 -> s7  [label="action:0.0"];
s0 -> s8  [label="action:0.03"];
s0 -> s9  [label="action:0.0"];
s0 -> s10  [label="action:0.04"];
s0 -> s11  [label="action:0.0"];
s0 -> s12  [label="action:0.03"];
s0 -> s13  [label="end:1.0"];
s1 -> s0  [label="init:1.0"];
s1 -> s2  [label="action:0.57"];
s1 -> s3  [label="action:0.08"];
s1 -> s4  [label="action:0.04"];
s1 -> s5  [label="action:0.08"];
s1 -> s

Model saved to learned_mdp_multiwoz_all.pdf.
